In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import os.path
import matplotlib.cm as cm

import tensorflow as tf
from tqdm import tqdm 

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image_dataset_from_directory

from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.applications import *

In [ ]:
train_dir = Path('../input/garbage-collection-dockship/garbage_classification_ai_challenge-dataset/TRAIN/')

test_dir = Path('../input/garbage-collection-dockship/garbage_classification_ai_challenge-dataset/TEST/')

In [ ]:
train_ds = image_dataset_from_directory(train_dir, labels = 'inferred', label_mode='categorical',color_mode='rgb', image_size=(299,299), batch_size=32, shuffle=True, validation_split=0.2, subset='training', seed=42)

val_ds = image_dataset_from_directory(train_dir, labels = 'inferred', label_mode='categorical',color_mode='rgb', image_size=(299,299), batch_size=32, shuffle=True,validation_split=0.2, subset='validation', seed=42)

test_ds = image_dataset_from_directory(test_dir, label_mode=None, image_size=(299,299), batch_size=1, shuffle=False, seed=42)

In [ ]:
data_augmentation = tf.keras.Sequential(
    [tf.keras.layers.experimental.preprocessing.Rescaling(1.0/149.5, offset= -1),tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),]
)

In [ ]:
base_model = Xception(weights="imagenet", input_shape=(299,299,3), include_top=False,)

In [ ]:
base_model.trainable = False

In [ ]:
inputs = Input(shape=(299,299,3))
x = data_augmentation(inputs)

x = base_model(x, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
outputs = Dense(6, activation='softmax')(x)

model = Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = Adam(), loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(train_ds,epochs=20, validation_data=val_ds)

In [ ]:
base_model.trainable = True
model.summary()

model.compile(
    optimizer = Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])



In [ ]:
checkpoint_path = 'models/NASNetLarge/Vgg16model_002.h5'

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss',mode='min', patience=30, verbose=1),
                ReduceLROnPlateau(monitor='val_loss', mode='min', factor=0.1, patience=10, min_lr=0.00000001, verbose=1),
                ModelCheckpoint(monitor='val_loss', mode='min', filepath=checkpoint_path,verbose=1, save_best_only=True, save_weights_only=False)
            ]

In [ ]:
history = model.fit(train_ds, epochs= 100, validation_data=val_ds, callbacks = callbacks)

In [ ]:
pd.DataFrame(history.history)[['accuracy', 'val_accuracy']].plot()
plt.title('Accuracy')
plt.show()

In [ ]:
pd.DataFrame(history.history)[['loss', 'val_loss']].plot()
plt.title('Loss')
plt.show()

In [ ]:
pred = model.predict(test_ds)
pred = np.argmax(pred,axis=1)

In [ ]:
pred

In [ ]:
d = {}
for index, class_name in enumerate(train_ds.class_names):
    d[index] = class_name

In [ ]:
pred_final = [d[k] for k in pred]
# for i, p in enumerate(pred_final):
#     print(f'{i} :: {p}')

In [ ]:
test_img_paths = list(test_dir.glob(r'**/*.jpg'))
test_img_names = list(map(lambda x: os.path.split(x)[1], test_img_paths))
test_img_names.sort()

In [ ]:
test_img_names = pd.Series(test_img_names, name='Filename').astype(str)
test_pred = pd.Series(pred_final,name='Labels')
final_df = pd.concat([test_img_names, test_pred], axis=1)

In [ ]:
final_df.to_csv('/kaggle/working/final15_csv.csv', index=False)

<a href="./final15_csv.csv"> Download File </a>